In [ ]:
import cmip6_cv.PrePARE.PrePARE as pp
import subprocess
import pandas as pd
import git 
from shutil import copy2

## 1. Settings. 

In [ ]:
trunk="/mnt/lustre02/work/ik1017/CMIP6/data/"
prepareSetting = {
    "exec" : pp.__file__ ,
    "logChunk":"/mnt/lustre02/work/ik1017/CMIP6/meta/tools/c34g-qc-prepare/logs/",
    "cmip6-cmor-table-path" : "/home/dkrz/k204210/cmip6-cmor-tables/Tables" }
dsetsinp = "/home/dkrz/k204210/catalog-to-manifest/catalogs/wg1subset-r1-datasets-pids-clean.csv.gz"
recentCV = "/home/dkrz/k204210/CMIP6-CV-20200617.json"

## 2. Initialize

In [ ]:
dsetslist = pd.read_csv(dsetsinp)
g = git.Git(prepareSetting["cmip6-cmor-table-path"]) 
g.reset("--hard")
g.checkout("master")
copy2(prepareSetting["cmip6-cmor-table-path"]+"/CMIP6_CV.json", recentCV)

Convert dataset to path to data in order to apply PrePARE on it

In [ ]:
def filepathFromDset(dset) :
    return trunk+dset["dataset_id"].replace('.','/')

In [ ]:
dsetslist["filepath"]=dsetslist.apply(lambda row: filepathFromDset(row), axis=1)

## 3. Prepare PrePARE

For applying PrePARE, we need the correct cmip6-cmor-tables release that matches the data_specs_version in the file.

In [ ]:
tags = reversed(g.tag("-n").split("\n"))
taglist = pd.DataFrame(columns=["data_specs_version","tag_label","description"])
for tag in tags :
    tl = tag.split(" ", 1)[0]
    tllen = len(tl.split("."))
    if tllen > 3 :
        continue
    dsvnumber = tl.split(".")[tllen-1]
    dsvnumber = "".join(filter(str.isdigit, dsvnumber))
    dsv = "['01.00."+dsvnumber+"']"
    if taglist[taglist["data_specs_version"] == dsv].empty :
        taglist = taglist.append({"data_specs_version":dsv,
                                  "tag_label":tl,
                                  "description":tag.split(" ",1)[1]},
                                 ignore_index=True)

In [ ]:
print(taglist)

## 4. Apply PrePARE

In [ ]:
def checkSubset(tocheck):
    check100 = 0
    for index, row in tocheck.iterrows():
        tag2checkout = taglist[taglist["data_specs_version"]==row["data_specs_version"]]["tag_label"].to_list()
        g.reset("--hard")
        g.checkout(tag2checkout)
        copy2(recentCV, prepareSetting["cmip6-cmor-table-path"]+"/CMIP6_CV.json")
        #
        logPath=prepareSetting["logChunk"]+row["data_specs_version"].split('.')[2].split("'")[0]+"/"+row["dataset_id"]
        a = subprocess.run([prepareSetting["exec"],
                        "-l",logPath,
                        "--table-path",prepareSetting["cmip6-cmor-table-path"],
                        row["filepath"]], capture_output=True)
        check100+=1
        if check100 == 100 :
            break

In [ ]:
for dsv in dsetslist["data_specs_version"].unique().tolist() :
    tocheck = dsetslist[dsetslist["data_specs_version"] == dsv]
    checkSubset(tocheck)